In [1]:
%cd ../
%matplotlib inline

/home/eli/AnacondaProjects/HTFATorch


In [2]:
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils

import csv
import glob
import logging
import json
import os
import re

import numpy as np
from sklearn.model_selection import StratifiedKFold

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

mask_file = '/home/eli/Documents/sound_depression/mask/sound_depression_mask_MNI152.nii.gz'

OVERWRITE = True
TASK_ONSET_DELAY = 3

In [4]:
original_db_file = 'data/sound_depression.db'
original_db = niidb.FMriActivationsDb(original_db_file, mask=mask_file, smooth=6)
num_blocks = len(list(original_db.all()))

xs = np.zeros([num_blocks, 1])
ys = []

for block in original_db.all():
    ys.append(block.run)

In [5]:
def write_db_file(k, train_indices):
    db_file = 'data/sound_depression_cv%d.db' % k
    existed = os.path.isfile(db_file)
    db = niidb.FMriActivationsDb(db_file, mask=mask_file, smooth=6)
    if not existed or OVERWRITE:
        for block in original_db.all():
            logging.info('Stimulus %s from %d to %d',
                         block.task, block.start_time,
                         block.end_time)
            if isinstance(block.individual_differences, str):
                block.individual_differences = json.loads(block.individual_differences)
            if isinstance(block.rest_start_times, str):
                block.rest_start_times = json.loads(block.rest_start_times)
            if isinstance(block.rest_end_times, str):
                block.rest_end_times = json.loads(block.rest_end_times)
            db.upsert(block, strip_id=True)

In [6]:
cv = StratifiedKFold(n_splits=3, shuffle=True)
for k, (train_indices, _) in enumerate(cv.split(xs, ys)):
    write_db_file(k, train_indices)

10/09/2020 14:49:35 Stimulus tones from 1 to 12
10/09/2020 14:49:35 Stimulus response from 12 to 13
10/09/2020 14:49:35 Stimulus negative_music from 13 to 23
10/09/2020 14:49:36 Stimulus response from 24 to 25
10/09/2020 14:49:36 Stimulus tones from 24 to 34
10/09/2020 14:49:36 Stimulus response from 35 to 36
10/09/2020 14:49:36 Stimulus positive_music from 36 to 46
10/09/2020 14:49:36 Stimulus response from 46 to 47
10/09/2020 14:49:36 Stimulus tones from 48 to 58
10/09/2020 14:49:36 Stimulus response from 58 to 59
10/09/2020 14:49:36 Stimulus negative_music from 59 to 69
10/09/2020 14:49:36 Stimulus response from 70 to 71
10/09/2020 14:49:36 Stimulus tones from 70 to 80
10/09/2020 14:49:36 Stimulus response from 81 to 82
10/09/2020 14:49:36 Stimulus positive_music from 82 to 92
10/09/2020 14:49:36 Stimulus response from 92 to 93
10/09/2020 14:49:36 Stimulus tones from 94 to 104
10/09/2020 14:49:36 Stimulus response from 104 to 105
10/09/2020 14:49:36 Stimulus positive_music from 13 t

In [7]:
logging.info('Finished building cross-validation NiiDbs out of Lepping 2017 dataset')

10/09/2020 14:52:10 Finished building cross-validation NiiDbs out of Lepping 2017 dataset
